Install all the cool stuff so that this thing would run (yeah you gotta install the legacy version cause the folks at Arizona won't bother updating everything to Tensorflow 2).

In [ ]:
from tealayer2 import Tea, AdditivePooling
from tensorflow.keras.layers import Flatten, Activation, Input, Lambda, concatenate
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model

import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.optimizers.legacy import Adam

from rancutils.teaconversion import create_cores, create_packets, Packet
from rancutils.output_bus import OutputBus
from rancutils.serialization import save as sim_save

Prepare your dataset. Note: X refers to the parameters used for prediction (in this case the pixel values), and Y refers to the actual prediction (in this case numbers from 0 to 9 inferred from the images).

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
            
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


Define your input (2D array of the coordinates of the pixel of an image, with only 1 value - greyscale value) and flatten them out.

In [ ]:
# Greyscale images are of shape (28,28,1)
inputs = Input(shape=(28,28,1))

# Flatten the inputs so that inputs map as: flatten_input[0] -> axon[0], ..., flatten_input[255] -> axon[255]
flattened_inputs = Flatten()(inputs)

Now comes the fun part. You get to define the topology of your RANC Network. The code below would create a network with the following attributes:
- The first layer of the network consists of 4 cores. Each core takes in 256 inputs from flattened_inputs. You may notice that the inputs taken by each core overlap with each other. This overlap is intentional and focused on the center of the image (where most of the bright pixels of the image are situated).
- Each core of the first layer spits out 64 outputs (or "neurons" in RANC lingo). These "neurons" are taken in by the second layer of the network. This second layer only has 250 outputs, which are split into 10 groups of 25 to predict the 10 classes from 0 to 9.
- How do we split the output of the second layer? By using the utility class AdditivePooling.

In [ ]:
# Generate each core.
# We are taking a 16x16 square of the input image and striding it by 12. this gives us 4 cores with 0 padding encumpassing the entire image.
core0 = Lambda(lambda x : x[:, :256])(flattened_inputs)
core1 = Lambda(lambda x : x[:, 176:432])(flattened_inputs)
core2 = Lambda(lambda x : x[:, 352:608])(flattened_inputs)
core3 = Lambda(lambda x : x[:, 528:])(flattened_inputs)

# Use the image distributions as corresponding inputs into our Tea Layer.
core0 = Tea(units=64, name='tea_1_1')(core0)
core1 = Tea(units=64, name='tea_1_2')(core1)
core2 = Tea(units=64, name='tea_1_3')(core2)
core3 = Tea(units=64, name='tea_1_4')(core3)

# The classification is the concatenation of these 4 core's outputs.
# We'll call the classification core our 'network'
network = concatenate([core0, core1, core2, core3])

network = Tea(units=250, name='tea_2')(network)

network = AdditivePooling(10)(network)

With all the preparations done, we can now start to train the network and test it.

In [ ]:
predictions = Activation('softmax')(network)

model = Model(inputs=inputs, outputs=predictions)

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

score = model.evaluate(X_test, y_test, verbose=0)

Print the score (or more accurately, the loss and the accuracy of the model). At the time of writing this, the accuracy should fall somewhere between 92-95% (which kinda sucks).

In [ ]:
print("Test Loss: ", score[0])
print("Test Accuracy: ", score[1])

Now, the crux - converting the inputs to input spikes to be passed into the C++ simulator and the outputs to output spikes to be used for cross-validation between the Tensorflow environment, C++ simulator, and FPGA implementation.

In case you don't know or wonder why, the point of RANC is to make advancement in neurophormic computing - to ultilize SNN and implement them effectively on dedicated hardware.

RANC is not created to be "yet another AI model" that is "0.01% percent more accurate than that ABCXYZ model published in a paper last month".

In [ ]:
x_test_flat = X_test.reshape((10000, 784))
partitioned_packets = []

# Use absolute/hard reset by specifying neuron_reset_type=0
cores_sim = create_cores(model, 2, neuron_reset_type=0)
num_test_samples = 100
# Partition the packets into groups as they will be fed into each of the input cores
partitioned_packets.append(x_test_flat[:num_test_samples, :256])
partitioned_packets.append(x_test_flat[:num_test_samples, 176:432])
partitioned_packets.append(x_test_flat[:num_test_samples, 352:608])
partitioned_packets.append(x_test_flat[:num_test_samples, 528:])
packets_sim = create_packets(partitioned_packets)
output_bus_sim = OutputBus((0, 2), num_outputs=250)

We create the input.json file for the C++ simulator.

In [ ]:
# This file can then be used as an input json to the RANC Simulator through the "input file" argument.
sim_save("./toy/mnist_config.json", cores_sim, packets_sim, output_bus_sim, indent=2)

We also save the prediction result of the Tensorflow environment, along with the correct result for later cross-validation.

In [ ]:
predict = model.predict(X_test[:num_test_samples,:])
print(type(predict))
idx = []
for i in predict:
  idx.append(np.argmax(i))
test_predictions = to_categorical(idx)

# Additionally, output the tensorflow predictions and correct labels for later cross validation
np.savetxt("./toy/mnist_tf_preds.txt", test_predictions, delimiter=',', fmt='%i')
np.savetxt("./toy/mnist_correct_preds.txt", y_test[0:99], delimiter=',', fmt='%i')

The above code saves the result in the form of one-hot. We can also save the result in spike form. 

In [ ]:
layer_name = model.layers[-3].name
cool_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
spike_output = cool_model.predict(X_test[:num_test_samples,:])
#print(intermediate_output)

np.savetxt('./toy/spike_output.txt', spike_output, fmt='%i')